In [1]:
!pip -q install bitsandbytes accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 20.7 MB/s eta 0:00:00


In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import re

In [2]:
model_id = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

In [3]:
llm = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200)

Device set to use cuda:0


In [4]:
def create_prompt(question, observation=None):
    prompt = f"""You are a helpful assistant that solves problems step by step using Python.
Question: {question}
Let's solve it step by step.
"""
    if observation:
        prompt += f"# Previous Observation: {observation}\n"
    prompt += "# Please write valid Python code and store the final answer in a variable named `result`.\n"
    prompt += "Thought:\n# Think about the problem.\nAction:\n"
    return prompt

In [5]:
def extract_code(text):
    match = re.search(r"Action:\n([\s\S]+)", text)
    if match:
        code = match.group(1).strip()
        lines = code.splitlines()
        code_lines = [line for line in lines if line.strip() and not line.strip().startswith("#")]
        return "\n".join(code_lines)
    return ""

In [6]:
def run_code(code):
    exec_env = {}
    try:
        exec(code, {}, exec_env)
    except Exception as e:
        print("Execution error:", e)
        return f"Execution Error: {e}"
    if "result" in exec_env:
        return exec_env["result"]
    nums = [v for v in exec_env.values() if isinstance(v, (int, float))]
    if nums:
        return nums[-1]
    return "No result variable found."


In [7]:
def pal_chain(question, max_steps=3):
    observation = None
    for step in range(1, max_steps + 1):
        print(f"\n--- Step {step} ---")
        prompt = create_prompt(question, observation)
        output = llm(prompt)[0]
        generated = output.get('generated_text', '') or output.get('text', '')
        code = extract_code(generated)
        if not code:
            print("No code extracted.")
            return "No code generated."
        print("Generated code:\n", code)
        observation = run_code(code)
        print("Observation:", observation)
        if isinstance(observation, (int, float, str)):
            break
    return observation


In [8]:

question = "Reza has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"

final_answer = pal_chain(question)
print("\n✅ Final Answer:", final_answer)


--- Step 1 ---
Generated code:
 first_tennis_balls = 5
new_tennis_balls = 2 * 3
result = first_tennis_balls + new_tennis_balls
print(f"Reza has {result} tennis balls now.")
Reza has 11 tennis balls now.
Observation: 11

✅ Final Answer: 11
